In [78]:
import pandas as pd
import os,natsort


In [79]:
e2e_dir = "/mydata/profiler_benchmark/e2e/"
raw_e2e_file = os.path.join(e2e_dir, "raw_e2e.csv")
normalized_e2e_file = os.path.join(e2e_dir, "normalized_e2e.csv")

In [80]:
# recursively find all the csv files in the directory
csv_files = []
for root, dirs, files in os.walk(e2e_dir):
    for file in files:
        if file.endswith(".log"):
            csv_files.append(os.path.join(root, file))


In [81]:
csv_files

['/mydata/profiler_benchmark/e2e/custom_log_b1024_gpu4.log',
 '/mydata/profiler_benchmark/e2e/scalene_b512_gpu1.log',
 '/mydata/profiler_benchmark/e2e/no_profiler_b1024_gpu4.log',
 '/mydata/profiler_benchmark/e2e/no_profiler_b512_gpu1.log',
 '/mydata/profiler_benchmark/e2e/py-spy_b512_gpu1.log',
 '/mydata/profiler_benchmark/e2e/custom_log_b512_gpu1.log',
 '/mydata/profiler_benchmark/e2e/py-spy_b1024_gpu4.log']

In [82]:
# convert all the csv files to pandas dataframes, add file name as a column and concat them
df = pd.concat((pd.read_csv(f).assign(file_name=os.path.basename(f)) for f in csv_files),ignore_index=True)


In [83]:
# save the dataframe to a csv file
df.to_csv(raw_e2e_file, index=False)

In [84]:
# sort by file name using natsort and reset the index

df = df.iloc[natsort.index_natsorted(df['file_name'])]
df = df.reset_index(drop=True)

In [85]:
#  get row number with file_name beginning with "no_profiler"
no_profiler_index = df[df['file_name'].str.startswith("no_profiler")].index.values[0]

In [86]:
# normalize every column, except "file_name" with respect to the no_profiler_index in % and subtract 100

# copy the dataframe
df_norm = df.copy()

for col in df_norm.columns:
    if col != "file_name":
        df_norm[col] = df_norm[col].div(df_norm.iloc[no_profiler_index][col])

        

In [87]:
# save the normalized dataframe to a csv file
df_norm.to_csv(normalized_e2e_file, index=False)

In [90]:
df_norm.head(len(df_norm))

,wall(s),user(s),kernel(s),max_rss(KB),file_name
0,1.001416,1.011553,1.062180,1.009760,custom_log_b512_gpu1.log
1,0.247356,1.048468,1.305170,3.059655,custom_log_b1024_gpu4.log
2,1.000000,1.000000,1.000000,1.000000,no_profiler_b512_gpu1.log
3,0.244783,1.038044,1.314219,3.060932,no_profiler_b1024_gpu4.log
4,1.085003,1.310344,5.404364,0.992813,py-spy_b512_gpu1.log
5,0.309684,0.262709,2.998675,0.149951,py-spy_b1024_gpu4.log
6,1.961125,1.272693,4.711632,1.121397,scalene_b512_gpu1.log


In [91]:
df.head(len(df))

,wall(s),user(s),kernel(s),max_rss(KB),file_name
0,9459.55,8334.30,713.70,4756408,custom_log_b512_gpu1.log
1,2336.57,8638.44,876.97,14412296,custom_log_b1024_gpu4.log
2,9446.17,8239.11,671.92,4710432,no_profiler_b512_gpu1.log
3,2312.26,8552.56,883.05,14418312,no_profiler_b1024_gpu4.log
4,10249.12,10796.07,3631.30,4676580,py-spy_b512_gpu1.log
5,2925.33,2164.49,2014.87,706332,py-spy_b1024_gpu4.log
6,18525.12,10485.86,3165.84,5282264,scalene_b512_gpu1.log
